# Feature Engineering and EDA

In [43]:
import re
import numpy as np
import pandas as pd
import pickle
import gensim
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
from gensim.models import CoherenceModel
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

%matplotlib inline

In [146]:
file = open('articles.p', 'rb')      
df = pickle.load(file)
file.close()

In [147]:
drop_indices = df.loc[df['date'] < pd.Timestamp(2019, 3, 15)].index
df.drop(index=drop_indices, inplace=True)

In [148]:
# Drop update articles and investing articles
df = df.loc[df['headline'].map(lambda x: re.search(r'UPDATE', x)).isna()]
df = df.loc[df['headline'].map(lambda x: re.search(r'US STOCKS', x)).isna()]
df = df.loc[df['url'].map(lambda x: re.search(r'investing', x)).isna()]
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)
df.drop(columns='url', inplace=True)

In [149]:
def replace_words(text):
    text = re.sub(r'U\.S\.', 'US', text)
    text = re.sub(r'U\.S\.A\.', 'US', text)
    text = re.sub(r'US', 'USA', text)
    text = re.sub(r'Kongs', 'Kong', text)
    text = re.sub(r'Hong Kong', 'HongKong', text)
    text = re.sub(r'U\.K\.', 'UK', text)
    text = re.sub(r'Mr\.', 'MR', text)
    text = re.sub(r'Mrs\.', 'MRS', text)
    text = re.sub(r'Ms\.', 'MS', text)
    text = re.sub(r'\.\.\.', '', text)
    text = re.sub(r'U.S-China', 'US-China', text)
    text = text.replace('Co.', 'Co')
    text = text.replace('\xa0', '')
    text = text.replace('."', '".')
    text = text.replace('immediatelywith', 'immediately with')
    text = text.replace('theOfficeof', 'the Office of')
    text = text.replace('theCommissionerof', 'the Commissioner of')
    return text

In [150]:
df['body'] = df['body'].map(replace_words)

In [161]:
def preprocess_words(text):
    stop_words = stopwords.words('english')
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words:
            result.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return result

In [162]:
def preprocess_sentence(text):
    stop_words = stopwords.words('english')
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words:
            result.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return ' '.join(result)

In [163]:
def preprocess_all(listy):
    results = [ preprocess_sentence(text) for text in listy ]
    return results

In [160]:
# Tokenize body sentences, keep first 10 sentences
df['sentence_tokens'] = df['body'].map(lambda x: sent_tokenize(x)[:10])

# Preprocess sentences
df['sentence_tokens'] = df['sentence_tokens'].map(preprocess_all)

# Preprocess words
df['word_tokens'] = df['body'].map(preprocess_words)

In [ ]:
dictionary = gensim.corpora.Dictionary(pro_body)

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [ dictionary.doc2bow(doc) for doc in pro_body ]

In [ ]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
lda_model = models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
mallet_path = 'Users/waynelam/Documents/DevStuff/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
def vader_analysis(text):
    analyzer = SentimentIntensityAnalyzer()
    return analyzer.polarity_scores(text)

In [ ]:
def polarity(text):
    analysis = TextBlob(text)
    return analysis.sentiment[0]

def subjectivity(text):
    analysis = TextBlob(text)
    return analysis.sentiment[1]